In [1]:
class Order:
    def __init__(self, Status, Category, Type, Size, Price, Index, Time):
        self.Status = Status # Add, Cancel, Outstanding, Executed
        self.Category = Category # Market, Limit, and for makrket order, price = 0 or max_price
        self.Type = Type # Bid, Ask
        self.Size = Size # Int
        self.Price = Price
        self.Index = Index 
        self.Time = Time
        self.ID = str(Status)[0]+str(Category)[0]+str(Type)[0]+str(Index)+str(Time)

    def content(self):
        return [self.Status, self.Category, self.Type, self.Size, self.Price, self.Index, self.Time, self.ID]

    def __repr__(self):
        return repr((self.Status, self.Category, self.Type, self.Size, self.Price, self.Index, self.Time))


class Order_pair:
    def __init__(self,order1,order2,price,size,time):
        
        self.Exe_order = order1
        self.Pair_order = order2
        self.Exe_price = price
        self.Exe_size = size
        self.Exe_time = time
        
    def content(self):
        return [self.Exe_order, self.Pair_order, self.Exe_price, self.Exe_size, self.Exe_time]

    def __repr__(self):
        return repr((self.Exe_order, self.Pair_order, self.Exe_price, self.Exe_size, self.Exe_time))


In [2]:
import copy
#player_pool = {Index:Trader}

class Market:
    def __init__(self,maxprice):
        self.Order_Pool = {}
        self.Bid_Orders = {}
        self.Ask_Orders = {}
        self.Ask_Price = 0
        self.Bid_Price = maxprice
        self.Bid_Dynamics = []
        self.Ask_Dynamics = []

        
                
        
    def Get_Ask_Price(self):
        asks = list(self.Ask_Orders.keys())
        if len(asks) > 0:
            self.Ask_Price = min(asks)
        
    def Get_Bid_Price(self):
        bids = list(self.Bid_Orders.keys())
        if len(bids) > 0:
            self.Bid_Price = max(bids)
        
    def Write_into_Pool(self,order):
        self.Order_Pool[order.ID] = order
        
    def Pool_to_Book(self):
        self.Bid_Orders = {}
        self.Ask_Orders = {}
        
        for ID in list(self.Order_Pool.keys()):
    
            order = self.Order_Pool[ID]
            if order.Type == 'Bid':
                if order.Price in list(self.Bid_Orders.keys()):
                    self.Bid_Orders[order.Price].append(order.ID)
                                
                else:
                    self.Bid_Orders[order.Price] = [order.ID]
                
            if order.Type == 'Ask':
                if order.Price in list(self.Ask_Orders.keys()):
                    self.Ask_Orders[order.Price].append(order.ID)
                                
                else:
                    self.Ask_Orders[order.Price] = [order.ID]
                    
        self.Get_Ask_Price()
        self.Get_Bid_Price()
        
    def Trade(self, order, time):
        trade_result = []

        remaining_size = order.Size
        orders = []
        if order.Type == 'Bid':
            optional_price = list(self.Ask_Orders.keys())
            optional_price.sort()
            for p in optional_price:
                if p <= order.Price:
                    orders += self.Ask_Orders[p]
        
        else:
            optional_price = list(self.Bid_Orders.keys())
            optional_price.sort(reverse = True)
            for p in optional_price:
                if p >= order.Price:
                    orders += self.Bid_Orders[p]
                    

        dead_ID = []
        for ID in orders:
            pair = self.Order_Pool[ID]
            order_c = copy.deepcopy(order)
            
            if pair.Size <= remaining_size:
                exe_price = pair.Price
                exe_size = pair.Size
                order_pair = Order_pair(order_c,pair,exe_price,exe_size,time)
                remaining_size -= exe_size
                trade_result.append(order_pair)
                dead_ID.append(ID)
                
            elif pair.Size > remaining_size and remaining_size > 0:
                exe_price = pair.Price
                exe_size = remaining_size
                remaining_size = 0
                pair.Size -= exe_size
                order_pair = Order_pair(order_c,pair,exe_price,exe_size,time)
                trade_result.append(order_pair)
                break
                
            else:
                break
                
        if dead_ID != []:
            for d in dead_ID:
                del self.Order_Pool[d]
                
        if remaining_size > 0:
            order.Size = remaining_size
            self.Write_into_Pool(order)
            
        return trade_result
        
        
        
    def Update(self,order_o,time):#order_o original order
        order = copy.deepcopy(order_o)
        results = 0
        if order != 0:
            if order.Status == 'Add':
                if order.Type == 'Bid':
                    if order.Price < self.Ask_Price:
                        self.Write_into_Pool(order)

                    else:
                        results = self.Trade(order,time)

                if order.Type == 'Ask':
                    if order.Price > self.Bid_Price:
                        self.Write_into_Pool(order)

                    else:
                        results = self.Trade(order,time)
                        
            if order.Status == 'Cancel':
                del self.Order_Pool[order.ID]
                
                
            self.Pool_to_Book()
            
        return results

In [3]:
import random
class Trader:
    def __init__(self, index, asset, cash, risk, loss):
        self.Index = index
        self.Asset = asset
        self.Cash = cash
        self.Available_asset = asset
        self.Available_cash = cash
        self.Risk_bear_level = risk
        self.Max_loss = loss
        self.Order_history = []
        self.Outstanding_bid = {}
        self.Outstanding_ask = {}
        self.Outstanding_order = {}
        self.Executed_order = []
        self.Events = []
        self.Surplus = 0
        
    def Pool_to_Order(self):
        self.Outstanding_bid = {}
        self.Outstanding_ask = {}
        for ID in list(self.Outstanding_order.keys()):
            order = self.Outstanding_order[ID]
            if order.Type == 'Bid':
                if order.Price in list(self.Outstanding_bid.keys()):
                    self.Outstanding_bid[order.Price].append(order.ID)
                                
                else:
                    self.Outstanding_bid[order.Price] = [order.ID]
                
            if order.Type == 'Ask':
                if order.Price in list(self.Outstanding_ask.keys()):
                    self.Outstanding_ask[order.Price].append(order.ID)
                                
                else:
                    self.Outstanding_ask[order.Price] = [order.ID]
                    
    def Place_Order(self,neworder):
        action = neworder.Status
        neworder.Index = self.Index
        self.Outstanding_order[neworder.ID] = neworder
        print('Trader ',self.Index,' place a ',neworder)


        if action == 'Cancel':
            return 0
        
        elif action == 'Add':
            if neworder.Category == 'Limit':
                self.Outstanding_order[neworder.ID] = neworder
                if neworder.Type == 'Ask':
                    self.Available_asset = self.Asset - neworder.Size
                    
                elif neworder.Type == 'Bid':
                    self.Available_cash = self.Cash - (neworder.Size * neworder.Price)
            self.Pool_to_Order()
            
            return neworder
        
    def Update(self, order_pair,position):
        if position == 0:
        
            order = order_pair.Exe_order
            
        elif position == 1:
            order = order_pair.Pair_order
            
        ID = order.ID

        self.Outstanding_order[ID].Size -= order_pair.Exe_size
        self.Surplus += abs(order.Price-order_pair.Exe_price)*order_pair.Exe_size

        dead_ID = []
        if self.Outstanding_order[ID].Size == 0:
            dead_ID.append(ID)
            
        if order.Type == 'Bid':
            self.Asset += order_pair.Exe_size
            self.Cash -= (order_pair.Exe_price*order_pair.Exe_size)
        
        elif order.Type == 'Ask':
            self.Asset -= order_pair.Exe_size
            self.Cash += (order_pair.Exe_price*order_pair.Exe_size)
            
        if dead_ID != []:
            for d in dead_ID:
                del self.Outstanding_order[d]
                
        self.Pool_to_Order()
        
            
    def Min_Ask_Price(self):
        asks = list(self.Outstanding_ask.keys())
        if asks == []:
            a = 100
        else:
            a = min(asks)
        return a
        
    def Max_Bid_Price(self):
        bids = list(self.Outstanding_bid.keys())
        if bids == []:
            b = 0
        else:
            b = max(bids)
        return b        
    
    def Get_Information(self):
        print('Trader ',self.Index,' holds ',self.Asset,' assets and ',self.Cash,' cash')
        print('It has outstanding orders listed ',self.Outstanding_order)
        print('Available ',self.Available_asset,self.Available_cash)    
        

